In [2]:
import sys
sys.path.append('../')
from optm_algorithms.pso import PSO
from optm_algorithms.differential_evolution import DifferentialEvolutionAlgorithm

from fitness_function import RocketFitness, bound_values, fitness_func
import numpy as np
import warnings
from tqdm import tqdm
import json


In [3]:
# Inicializando função de fitness
rocket_fitness = RocketFitness(bound_values, num_workers=4)
random_values = np.random.rand(10,10)
fitness_func_class = rocket_fitness.calc_fitness


# Análise de Sensibilidade do PSO

Hiperparâmetros a serem otimizados:
- global_factor
- speed_factor
- v_max

Condições:
- local_factor + global_factor > 4
- todos os hiperparâmetros são maiores que zero

In [4]:
global_factor_list = np.linspace(1, 6, 5)
local_factor_list = np.linspace(1, 6, 5)
v_max = np.linspace(1,10,5)
# Printa hiperparâmetros
print("Global factor list: ", global_factor_list)
print("Local factor list: ", local_factor_list)
print("V_max: ", v_max)

grid1, grid2, grid3 = np.meshgrid(global_factor_list, local_factor_list, v_max)

combinations = np.vstack((grid1.ravel(), grid2.ravel(), grid3.ravel())).T
sum_of_columns = combinations[:, 0] + combinations[:, 1]

# Use logical indexing to select rows where the sum of the first and second columns is greater than 4
combinations = combinations[sum_of_columns > 4]


Global factor list:  [1.   2.25 3.5  4.75 6.  ]
Local factor list:  [1.   2.25 3.5  4.75 6.  ]
V_max:  [ 1.    3.25  5.5   7.75 10.  ]


In [9]:
combinations.shape

(110, 3)

In [5]:

def execute_sensitivity_analysis_pso(combinations, filename):
    
    #combinations = [[2.05, 2.05, 1],[2.05, 2.05, 1]]

    simulations_list = []
    for row in tqdm(combinations):
        
        global_factor = row[0]
        local_factor = row[1]
        v_max = row[2]
        
        pso_alg = PSO(
            num_epochs=100,
            pop_size=1000,
            chrom_length=10,
            n_best=2,
            global_factor = global_factor,
            local_factor = local_factor,
            speed_factor = 1,
            v_max=v_max,
            value_ranges=bound_values,
            fitness_func=fitness_func_class,
            neighborhood_mode='self',
            verbose=True,
            eval_every=10,
            )

        best_solutions = pso_alg.fit()

        dict_save = {'global_factor': global_factor,
                     'local_factor': local_factor,
                     'v_max': v_max,
                     'fitness_calls': pso_alg.fitness_calls_list.tolist(),
                     'best_ind_list': pso_alg.best_ind_list.tolist(),
                     'avg_ind_list': pso_alg.avg_ind_list.tolist(),
                     'best_solutions': best_solutions.tolist(),
                     'total_time': pso_alg.total_exec_time,
                     }
        simulations_list.append(dict_save)
        
    with open(filename, 'w') as fout:
        json.dump(simulations_list, fout)

    return
        

execute_sensitivity_analysis_pso(combinations, 'simulations/pso_sensitivity.json')

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch 0: Best: 0.5688369478323626, Average: 0.24442910624309014


Epoch 10: Best: 0.6343523979810934, Average: 0.5029435026480478


Epoch 20: Best: 0.6371034360539911, Average: 0.5490170098417358


  0%|          | 0/110 [00:15<?, ?it/s]


KeyboardInterrupt: 

# Análise de Sensibilidade da Evol ução Diferencial

Hiperparâmetros a serem otimizados:
- mutation_rate
- crossover_rate

Condições:
- mutation_rate e crossover_rate <=1


In [6]:
mutation_rate = np.linspace(0.1, 1, 10)
crossover_rate = np.linspace(0.1, 1,10)
print("Mutation rate: ", mutation_rate)
print("Crossover rate: ", crossover_rate)
grid1, grid2 = np.meshgrid(mutation_rate, crossover_rate)
combinations = np.vstack((grid1.ravel(), grid2.ravel())).T

Mutation rate:  [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
Crossover rate:  [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


In [8]:

def execute_sensitivity_analysis_de(combinations, filename):
    
    #combinations = [[0.5, 0,5]]

    simulations_list = []
    for row in tqdm(combinations):
        
        mutation_rate = row[0]
        crossover_rate = row[1]
        
        de_alg = DifferentialEvolutionAlgorithm(
                                            num_epochs=100,
                                            pop_size=1000,
                                            chrom_length=10,
                                            value_ranges=bound_values,
                                            mutation_rate=mutation_rate,
                                            crossover_rate=crossover_rate,
                                            fitness_func=fitness_func_class,
                                            verbose=True,
                                            eval_every=10,
                                            )

        best_solutions = de_alg.fit()

        dict_save = {'global_factor': mutation_rate,
                     'local_factor': crossover_rate,
                     'fitness_calls': de_alg.fitness_calls_list.tolist(),
                     'best_ind_list': de_alg.best_ind_list.tolist(),
                     'avg_ind_list': de_alg.avg_ind_list.tolist(),
                     'best_solutions': best_solutions.tolist(),
                     'total_time': de_alg.total_exec_time,
                     }
        simulations_list.append(dict_save)
        
    with open(filename, 'w') as fout:
        json.dump(simulations_list, fout)

    return
        

execute_sensitivity_analysis_de(combinations, 'simulations/de_sensitivity.json')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0: Best: 0.5688369478323626, Average: 0.2834090829919377


  0%|          | 0/100 [00:06<?, ?it/s]


KeyboardInterrupt: 